In [8]:
from scipy.io.wavfile import read, write
import torchaudio
import torch
from librosa.util import normalize
from librosa.filters import mel as librosa_mel_fn
import numpy as np
import librosa
from IPython.display import Audio
from tqdm import tqdm
import os
import soundfile as sf
import json

In [9]:
MAX_WAV_VALUE = 32768.0

def load_wav(full_path):
    sampling_rate, data = read(full_path)
    return data, sampling_rate

def dynamic_range_compression(x, C=1, clip_val=1e-5):
    return np.log(np.clip(x, a_min=clip_val, a_max=None) * C)

def dynamic_range_decompression(x, C=1):
    return np.exp(x) / C

def dynamic_range_compression_torch(x, C=1, clip_val=1e-5):
    return torch.log(torch.clamp(x, min=clip_val) * C)

def dynamic_range_decompression_torch(x, C=1):
    return torch.exp(x) / C

def spectral_normalize_torch(magnitudes):
    output = dynamic_range_compression_torch(magnitudes)
    return output

def spectral_de_normalize_torch(magnitudes):
    output = dynamic_range_decompression_torch(magnitudes)
    return output

mel_basis = {}
hann_window = {}

def mel_spectrogram(y, n_fft, num_mels, sampling_rate, hop_size, win_size, fmin, fmax, center=False):
    if torch.min(y) < -1.:
        print('min value is ', torch.min(y))
    if torch.max(y) > 1.:
        print('max value is ', torch.max(y))

    global mel_basis, hann_window
    if fmax not in mel_basis:
        mel = librosa_mel_fn(sr=sampling_rate, n_fft=n_fft, n_mels=num_mels, fmin=fmin, fmax=fmax)
        mel_basis[str(fmax)+'_'+str(y.device)] = torch.from_numpy(mel).float().to(y.device)
        hann_window[str(y.device)] = torch.hann_window(win_size).to(y.device)

    y = torch.nn.functional.pad(y.unsqueeze(1), (int((n_fft-hop_size)/2), int((n_fft-hop_size)/2)), mode='reflect')
    y = y.squeeze(1)

    spec = torch.stft(y, n_fft, hop_length=hop_size, win_length=win_size, window=hann_window[str(y.device)],
                      center=center, pad_mode='reflect', normalized=False, onesided=True)

    spec = torch.sqrt(spec.pow(2).sum(-1)+(1e-9))

    spec = torch.matmul(mel_basis[str(fmax)+'_'+str(y.device)], spec)
    spec = spectral_normalize_torch(spec)

    return spec

In [10]:
with open("/home/v-yuancwang/AUDIT_v2/medata_infos/fsd_short.json", "r") as f:
    fsd_short_infos = json.load(f)
with open("/home/v-yuancwang/AUDIT_v2/medata_infos/fsd50k_short.json", "r") as f:
    fsd50k_short_infos = json.load(f)
b_wav_infos = fsd_short_infos + fsd50k_short_infos
print(len(b_wav_infos))

35044


In [11]:
ac_train_json = "/home/v-yuancwang/AUDIT_v2/medata_infos/ac_train.json"
ac_val_json = "/home/v-yuancwang/AUDIT_v2/medata_infos/ac_val.json"
audioset_sl_json = "/home/v-yuancwang/AUDIT_v2/medata_infos/audioset_sl.json"
bbc_json = "/home/v-yuancwang/AUDIT_v2/medata_infos/bbc.json"
soundbible_json = "/home/v-yuancwang/AUDIT_v2/medata_infos/soundbible.json"
fsd_10s_json = "/home/v-yuancwang/AUDIT_v2/medata_infos/fsd_10s.json"
vgg_json = "/home/v-yuancwang/AUDIT_v2/medata_infos/vggsound.json"

with open(ac_train_json, "r") as f:
    ac_train_infos = json.load(f)
with open(ac_val_json, "r") as f:
    ac_val_infos = json.load(f)
with open(audioset_sl_json, "r") as f:
    audioset_sl_infos = json.load(f)
with open(bbc_json, "r") as f:
    bbc_infos = json.load(f)
with open(soundbible_json, "r") as f:
    soundbible_infos = json.load(f)
with open(fsd_10s_json, "r") as f:
    fsd_10s_infos = json.load(f)
with open(vgg_json, "r") as f:
    vgg_infos = json.load(f)
np.random.shuffle(vgg_infos)
vgg_infos = vgg_infos[:10*1000]

print(len(ac_train_infos), len(ac_val_infos), len(audioset_sl_infos),
      len(bbc_infos), len(soundbible_infos), len(fsd_10s_infos))

seven_sets_infos = bbc_infos + soundbible_infos + fsd_10s_infos + audioset_sl_infos + ac_train_infos + ac_val_infos + vgg_infos
a_infos = []
for info in tqdm(seven_sets_infos[:]):
    if len(info["caption"].split(" ")) <= 10:
        a_infos.append(info)
print(len(a_infos))

45435 449 79147 11754 1232 13732


100%|██████████| 161749/161749 [00:00<00:00, 1008743.63it/s]

112044


In [13]:
np.random.shuffle(a_infos)
np.random.shuffle(b_wav_infos)

In [14]:
json_lists = []
remove_json_lists = []
replace_in_wav_path = "/blob/v-yuancwang/AUDITDATA/replace/in/wav"
replace_in_mel_path = "/blob/v-yuancwang/AUDITDATA/replace/in/mel"
replace_out_wav_path = "/blob/v-yuancwang/AUDITDATA/replace/out/wav"
replace_out_mel_path = "/blob/v-yuancwang/AUDITDATA/replace/out/mel"

In [15]:
replace_templates = ["Replace \{A\} with \{B\}", "Replace \{A\} to \{B\}",
                     "Swap out \{A\} with \{B\}", "Swap \{A\} with \{B\}",
                     "Exchange \{A\} with \{B\}", "Substitute \{A\} with \{B\}",
                     "Replace \{A\} with a new audio of \{B\}",
                     "Swap \{A\} with a new sound of \{B\}"]

In [17]:
json_lists = []
for i in tqdm(range(30000)[:]):

    a_id = np.random.randint(0, len(a_infos))
    b_id = np.random.randint(0, len(b_wav_infos))
    c_id = np.random.randint(0, len(b_wav_infos))
    
    a_wav_path, a_caption = a_infos[a_id]["mel"].replace("/mel/", "/wav/").replace(".npy", ".wav"), a_infos[a_id]["caption"]
    b_wav_path, b_caption = b_wav_infos[b_id]["wav"], b_wav_infos[b_id]["caption"]
    c_wav_path, c_caption = b_wav_infos[c_id]["wav"], b_wav_infos[c_id]["caption"]
    # print(a_caption, b_caption)

    template = np.random.choice(replace_templates)
    caption = template.replace("\{A\}", b_caption.lower().replace(".", "")).replace("\{B\}", c_caption.lower().replace(".", ""))

    a_wav, sr = librosa.load(a_wav_path, sr=16000)
    b_wav, sr = librosa.load(b_wav_path, sr=16000)
    b_wav = b_wav[: min(len(b_wav), int(16000*3.5))]
    c_wav, sr = librosa.load(c_wav_path, sr=16000)
    c_wav = c_wav[: min(len(c_wav), int(16000*3.5))]

    a_wav = np.pad(a_wav, (0, max(0, 16000*10-len(a_wav))), 'wrap')[:16000*10]
    pre_pad_length = np.random.randint(16000*3, int(16000*6.5))
    b_wav = np.pad(b_wav, (pre_pad_length, max(0, 16000*10-len(b_wav)-pre_pad_length)), 'constant', constant_values=(0, 0))[:16000*10]
    c_wav = np.pad(c_wav, (pre_pad_length, max(0, 16000*10-len(c_wav)-pre_pad_length)), 'constant', constant_values=(0, 0))[:16000*10]

    d_wav = a_wav + b_wav
    d_wav = np.clip(d_wav, -1, 1)
    e_wav = a_wav + c_wav
    e_wav = np.clip(e_wav, -1, 1)

    x = torch.FloatTensor(d_wav)
    x = mel_spectrogram(x.unsqueeze(0), n_fft=1024, num_mels=80, sampling_rate=16000,
                        hop_size=256, win_size=1024, fmin=0, fmax=8000)
    d_spec = x.cpu().numpy()[0]
    d_wav = d_wav * MAX_WAV_VALUE
    d_wav = d_wav.astype('int16')
    write(os.path.join(replace_in_wav_path, "{}".format(str(i))+".wav"), 16000, d_wav)
    np.save(os.path.join(replace_in_mel_path, "{}".format(str(i))+".npy"), d_spec)

    x = torch.FloatTensor(e_wav)
    x = mel_spectrogram(x.unsqueeze(0), n_fft=1024, num_mels=80, sampling_rate=16000,
                        hop_size=256, win_size=1024, fmin=0, fmax=8000)
    e_spec = x.cpu().numpy()[0]
    e_wav = e_wav * MAX_WAV_VALUE
    e_wav = e_wav.astype('int16')
    write(os.path.join(replace_out_wav_path, "{}".format(str(i))+".wav"), 16000, e_wav)
    np.save(os.path.join(replace_out_mel_path, "{}".format(str(i))+".npy"), e_spec)

    json_lists.append({"in_mel":os.path.join(replace_in_mel_path, "{}".format(str(i))+".npy"),
                       "out_mel":os.path.join(replace_out_mel_path, "{}".format(str(i))+".npy"),
                       "caption":caption})
 
with open("/home/v-yuancwang/AUDIT_v2/editing_medata_infos/replace.json", "w") as f:
    json.dump(json_lists, f)

  0%|          | 0/30000 [00:00<?, ?it/s]/opt/conda/envs/control/lib/python3.8/site-packages/torch/functional.py:632: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:801.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]
  0%|          | 35/30000 [00:13<2:46:40,  3.00it/s]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
in_mel_path = "/blob/v-yuancwang/AUDITDATA/replace/in/mel/8.npy"
out_mel_path = "/blob/v-yuancwang/AUDITDATA/replace/out/mel/8.npy"
in_mel = np.load(in_mel_path)
out_mel = np.load(out_mel_path)

In [ ]:
Audio(in_mel_path.replace("/mel/", "/wav/").replace(".npy", ".wav"))

In [ ]:
Audio(out_mel_path.replace("/mel/", "/wav/").replace(".npy", ".wav"))

In [ ]:
plt.imshow(in_mel)

In [ ]:
plt.imshow(out_mel)